In [ ]:
from textblob import TextBlob
import azure.cognitiveservices.speech as speechsdk
import tkinter as tk
from tkinter import messagebox, Text

# Pastikan Anda telah menginstal TextBlob dan nltk dengan pip
# pip install textblob
# python -m textblob.download_corpora

# Fungsi untuk menghapus teks dari kotak teks
def clear_text():
    text_box.delete("1.0", tk.END)

# Fungsi pengenalan ucapan
def recognize_speech():
    # Setel kunci dan wilayah dari sumber daya Azure Speech Anda
    speech_key = "5544cd8ec2fb442c8527470731b0daae"
    region = "southeastasia"

    # Buat objek konfigurasi dengan kunci dan wilayah
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=region)

    # Atur bahasa pengenalan ke Bahasa Indonesia
    speech_config.speech_recognition_language = "id-ID"

    # Buat pengenal ucapan dengan konfigurasi
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

    # Mulai pengenalan ucapan
    print("Mulai berbicara...")
    result = speech_recognizer.recognize_once()

    # Cetak hasil pengenalan ucapan
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        text_box.insert(tk.END, result.text)
    elif result.reason == speechsdk.ResultReason.NoMatch:
        messagebox.showinfo("Hasil", "Tidak ada ucapan yang terdeteksi.")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        messagebox.showerror("Error", "Pengenalan ucapan dibatalkan: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            messagebox.showerror("Error", "Kesalahan: {}".format(
                cancellation_details.error_details))

# Fungsi untuk analisis sentimen
def analyze_sentiment():
    # Ambil teks dari kotak teks
    text_to_analyze = text_box.get("1.0", tk.END)
    
    # Analisis sentimen menggunakan TextBlob
    blob = TextBlob(text_to_analyze)
    sentiment_score = blob.sentiment.polarity

    # Tentukan sentimen
    if sentiment_score > 0:
        sentiment = "Positif"
    elif sentiment_score < 0:
        sentiment = "Negatif"
    else:
        sentiment = "Netral"

    # Tampilkan hasil analisis sentimen
    messagebox.showinfo("Analisis Sentimen", "Sentimen teks ini adalah: " + sentiment)

# Membuat window utama
root = tk.Tk()
root.title("Azure Speech Recognition")

# Menambahkan kotak teks
text_box = Text(root, height=10, width=50)
text_box.pack(pady=(20, 10))

# Membuat tombol untuk menghapus teks
clear_button = tk.Button(root, text="Hapus Teks", command=clear_text, font=("Helvetica", 16), bg="#FF6347", fg="#ffffff")
clear_button.pack(pady=10)

# Membuat tombol untuk pengenalan ucapan
start_button = tk.Button(root, text="Mulai Pengenalan Ucapan", command=recognize_speech, font=("Helvetica", 16), bg="#34A2FE", fg="#ffffff")
start_button.pack(pady=20)

# Membuat tombol untuk analisis sentimen
sentiment_button = tk.Button(root, text="Analisis Sentimen", command=analyze_sentiment, font=("Helvetica", 16), bg="#90EE90", fg="#ffffff")
sentiment_button.pack(pady=10)

# Menjalankan aplikasi GUI
root.mainloop()
